# Brazil's Covid-19 spread


### Introduction
    
   Covid-19 is a virus originated from china. It's impact on the world is unquestionable. This notebook will try to analyse it's propagation in Brazil comparing with Italy, where the viruses impact is, by the time this notebook is written, more chaotic. <br>
   In order to help people and prevent this disease from spreading endlessly, I will be comparing both countries, using Foursquare api for mapping and intregrating it to the dateset already imported to maximize our accuracy of how would the virus spread in Brazil's territory, then we will be capable of elaborating solutions.
    

## Dataset

    I'm going to be using a realtime dataset from tableau. available on: 
    https://data.world/covid-19-data-resource-hub/covid-19-case-counts/workspace/file?filename=COVID-19+Cases.csv
    
### features

1. Date
 * Date of infection
 * type: Date
2. Country_Region
 * Country
 * type: String 
3. Province_State
 * Province or state
 * type: String
4. Case_Type
 * Status:
       1. Active
       2. Confirmed
       3. Deaths
       4. Recovered
 * type: String
5. Cases
 * type: Int
6. Difference
 * type: Int
7. Prep_Flow_Runtime
 * Dataset insert datetime
 * type: Datetime
8. Latest_Date
 * Latest insert date
 * type: Date
9. Lat
 * Latitude
 * type: Decimal
10. Long
 * Longitude
 * type: Decimal


### Data import

In [1]:
# Imports
import pandas as pd
import numpy as np


In [2]:
covid_df = pd.read_csv("https://query.data.world/s/hxqi5dd2vp5gk6af6cnitl7tm7remi")
covid_df.head()

,Date,Country_Region,Province_State,Case_Type,Cases,Difference,Prep_Flow_Runtime,Latest_Date,Lat,Long
0,3/11/2020,US,Arizona,Recovered,0,0,3/18/2020 5:57:42 AM,3/17/2020,32.0575,-111.6661
1,3/11/2020,US,Arizona,Deaths,0,0,3/18/2020 5:57:42 AM,3/17/2020,32.0575,-111.6661
2,3/11/2020,US,Arizona,Confirmed,0,0,3/18/2020 5:57:42 AM,3/17/2020,32.0575,-111.6661
3,3/11/2020,US,Arizona,Active,0,0,3/18/2020 5:57:42 AM,3/17/2020,32.0575,-111.6661
4,2/15/2020,US,Arizona,Recovered,0,0,3/18/2020 5:57:42 AM,3/17/2020,32.8162,-111.2845
